In [1]:
from matplotlib import pyplot as plt
import nlopt
import numpy as np
import kinpy as kp
import os

from omniisaacgymenvs.data_types import geometry_utils
from omniisaacgymenvs.data_types import se3

/home/wenzhaol/repos/OmniIsaacGymEnvs/omniisaacgymenvs/data_types/transformations.py:2021: UserWarning: No module named 'omniisaacgymenvs.data_types._transformations'
  warnings.warn(str(exc))


In [2]:
#urdf_path = '~/Downloads/data/hand/aSampleForearmURDF/aSampleForearm.urdf'
left_right = 'left'
urdf_path = '~/Downloads/data/hand/psyonic/ability_hand_' + left_right + '.urdf'
chain = kp.build_chain_from_urdf(open(os.path.expanduser(urdf_path), 'rb').read())
tip_joints = {
  'thumb_anchor_frame': ['thumb_q1', 'thumb_q2'],
  'index_anchor_frame': ['index_q1', 'index_q2'],
  'middle_anchor_frame': ['middle_q1', 'middle_q2'],
  'ring_anchor_frame': ['ring_q1', 'ring_q2'],
  'pinky_anchor_frame': ['pinky_q1', 'pinky_q2']
}
joint_indices = {name: index for index, name in enumerate(chain.get_joint_parameter_names())}
finger_tips = [
  'thumb_anchor_frame',
  'index_anchor_frame',
  'middle_anchor_frame',
  'ring_anchor_frame',
  'pinky_anchor_frame'
]
finger_chains = [kp.chain.SerialChain(
  chain, tip_frame, 'thumb_base_frame') for tip_frame in finger_tips]
mimic_config = {
  'index_q2': ('index_q1', 1.05851325, 0.72349796),
  'middle_q2': ('middle_q1', 1.05851325, 0.72349796),
  'ring_q2': ('ring_q1', 1.05851325, 0.72349796),
  'pinky_q2': ('pinky_q1', 1.05851325, 0.72349796),
}
mimic_joints = {}
for name, source in mimic_config.items():
  mimic_joints[joint_indices[name]] = (joint_indices[source[0]], source[1], source[2])
d_vector_d_tip_positions = np.array([
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [0, 0, 0, 0, 1],
  [-1, 1, 0, 0, 0],
  [-1, 0, 1, 0, 0],
  [-1, 0, 0, 1, 0],
  [-1, 0, 0, 0, 1],
  [0, -1, 1, 0, 0],
  [0, -1, 0, 1, 0],
  [0, -1, 0, 0, 1],
  [0, 0, -1, 1, 0],
  [0, 0, -1, 0, 1],
  [0, 0, 0, -1, 1]
]
)

def add_debug_frame(frame: se3.Transform,
                    parent_body: int = -1,
                    parent_link: int = -1,
                    line_length: float = 0.5,
                    line_width: float = 3,
                    duration: float = 100,
                    ):
  eye = np.identity(3)
  vec_start = frame.translation
  for i in range(3):
    trans = se3.Transform(xyz=line_length * eye[i])
    vec_end = (frame * trans).translation
    p.addUserDebugLine(vec_start, vec_end,
                        lineColorRGB=eye[i],
                        parentObjectUniqueId=parent_body,
                        parentLinkIndex=parent_link,
                        lineWidth=line_width,
                        lifeTime=duration)

Unknown attribute "name" in /robot[@name='ability_hand']/link[@name='base']/visual[1]
Unknown attribute "name" in /robot[@name='ability_hand']/link[@name='thumb_base']/visual[1]
Unknown attribute "name" in /robot[@name='ability_hand']/link[@name='index_L1']/visual[1]
Unknown attribute "name" in /robot[@name='ability_hand']/link[@name='index_L2']/visual[1]
Unknown attribute "name" in /robot[@name='ability_hand']/link[@name='middle_L1']/visual[1]
Unknown attribute "name" in /robot[@name='ability_hand']/link[@name='middle_L2']/visual[1]
Unknown attribute "name" in /robot[@name='ability_hand']/link[@name='ring_L1']/visual[1]
Unknown attribute "name" in /robot[@name='ability_hand']/link[@name='ring_L2']/visual[1]
Unknown attribute "name" in /robot[@name='ability_hand']/link[@name='pinky_L1']/visual[1]
Unknown attribute "name" in /robot[@name='ability_hand']/link[@name='pinky_L2']/visual[1]
Unknown attribute "name" in /robot[@name='ability_hand']/link[@name='thumb_L1']/visual[1]
Unknown attr

In [3]:
unit_length = 0.09473151311686484
def vector_length(vector):
  return np.linalg.norm(vector)

def normalize(vector):
  return vector / np.clip(vector_length(vector), 1e-10, 1e10)
  
def human_points_to_vectors(points):
  vectors = [
    points[4, :] - points[0, :],
    points[8, :] - points[0, :],
    points[12, :] - points[0, :],
    points[16, :] - points[0, :],
    points[20, :] - points[0, :],
    points[8, :] - points[4, :],
    points[12, :] - points[4, :],
    points[16, :] - points[4, :],
    points[20, :] - points[4, :],
    points[12, :] - points[8, :],
    points[16, :] - points[8, :],
    points[20, :] - points[8, :],
    points[16, :] - points[12, :],
    points[20, :] - points[12, :],
    points[20, :] - points[16, :],
  ]
  return vectors

def augment_reduced_joints(reduced_joint_positions):
  return np.array([0, 0, 0, 0, 0, 0,
                   reduced_joint_positions[0],
                   reduced_joint_positions[0] * 1.05851325 + 0.72349796,
                   reduced_joint_positions[1],
                   reduced_joint_positions[1] * 1.05851325 + 0.72349796,
                   reduced_joint_positions[2],
                   reduced_joint_positions[2] * 1.05851325 + 0.72349796,
                   reduced_joint_positions[3],
                   reduced_joint_positions[3] * 1.05851325 + 0.72349796,
                   reduced_joint_positions[4],
                   reduced_joint_positions[5]])

def robot_config_to_vectors(joint_positions):
  tip_positions = []
  for index, finger_tip_frame in enumerate(finger_tips):
    finger_joints = [joint_positions[joint_indices[joint_name]]
                     for joint_name in tip_joints[finger_tip_frame]]
    tip_positions.append(finger_chains[index].forward_kinematics(finger_joints).pos / unit_length)

  vectors = [
    tip_positions[0],
    tip_positions[1],
    tip_positions[2],
    tip_positions[3],
    tip_positions[4],
    tip_positions[1] - tip_positions[0],
    tip_positions[2] - tip_positions[0],
    tip_positions[3] - tip_positions[0],
    tip_positions[4] - tip_positions[0],
    tip_positions[2] - tip_positions[1],
    tip_positions[3] - tip_positions[1],
    tip_positions[4] - tip_positions[1],
    tip_positions[3] - tip_positions[2],
    tip_positions[4] - tip_positions[2],
    tip_positions[4] - tip_positions[3],    
  ]
  return vectors

def update_jacobian_for_mimic_joints(jacobian, mimic_joints):
  for joint_index, source in mimic_joints.items():
    source_index, multiplier, _ = source
    jacobian[:, source_index] = jacobian[:, joint_index] * multiplier + jacobian[:, source_index]
    jacobian[:, joint_index] = 0
  return jacobian

def compute_jacobians(joint_positions):
  jacobians = []
  for index, finger_tip_frame in enumerate(finger_tips):
    finger_joint_indices = [joint_indices[joint_name]
                            for joint_name in tip_joints[finger_tip_frame]]
    finger_joints = joint_positions[finger_joint_indices]
    jac = np.zeros((6, len(joint_positions)))
    jac[:, finger_joint_indices] = finger_chains[index].jacobian(finger_joints)
    update_jacobian_for_mimic_joints(jac, mimic_joints)
    jacobians.append(jac)
  return jacobians

In [4]:
import pybullet as p
import pybullet_data

physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(os.path.expanduser('~/Downloads/data/hand/psyonic')) #used by loadURDF
p.setGravity(0,0,-10)
robot_id = p.loadURDF(
  os.path.expanduser('~/Downloads/data/hand/psyonic/ability_hand_' + left_right + '.urdf'),
  [0,0,0], p.getQuaternionFromEuler([0,0,0]), useFixedBase=True)
p.resetDebugVisualizerCamera(cameraDistance=1.0, cameraYaw=200.8, cameraPitch=-57.4, cameraTargetPosition=[0.09, -0.28, -0.54])

xyzw = p.getBasePositionAndOrientation(robot_id)[1]
world_t_base = se3.Transform(xyz=p.getBasePositionAndOrientation(robot_id)[0], rot=[xyzw[3], xyzw[0], xyzw[1], xyzw[2]])

idx = 6
xyzw = p.getLinkState(robot_id, idx)[5]
base_t_thumb_base = world_t_base.inverse() * se3.Transform(xyz=p.getLinkState(robot_id, idx)[4], rot=[xyzw[3], xyzw[0], xyzw[1], xyzw[2]])
# add_debug_frame(base_t_thumb_base, robot_id, line_length=0.05, line_width=0.02)
# for idx in [9, 12, 15, 21]:
#   xyzw = p.getLinkState(robot_id, idx)[5]
#   world_t_link = world_t_base.inverse() * se3.Transform(xyz=p.getLinkState(robot_id, idx)[4], rot=[xyzw[3], xyzw[0], xyzw[1], xyzw[2]])
#   add_debug_frame(world_t_link, robot_id, line_length=0.03, line_width=0.03)

pybullet build time: May 20 2022 19:43:01


In [17]:
if left_right == "left":
  human_points = np.array([
## 17-00
[0.2730556130409241,0.7563595771789551,0.5648466348648071], 
[-0.1192198395729065,0.6576731204986572,0.30637118220329285], 
[-0.38098204135894775,0.44005051255226135,0.10161799192428589],
[-0.6844912767410278,0.1205785870552063,-0.11113680899143219], 
[-0.9271906614303589,-0.21813249588012695,-0.08076748996973038],            
[-0.22042199969291687,-0.015781765803694725,0.051046814769506454],
[-0.3627031147480011,-0.38654497265815735,-0.05687960982322693],           
[-0.4688083529472351,-0.6892058849334717,-0.13750480115413666],       
[-0.46379685401916504,-0.9809485673904419,-0.1895766705274582],                                                                                                                 
[-2.712847724239964e-10,-1.0111111947708196e-10,8.485199209928851e-10],  
[-0.06584032624959946,-0.4100193381309509,-0.13923749327659607],                                                                                                                
[-0.155704528093338,-0.7314178943634033,-0.22977814078330994],  
[-0.05695474520325661,-1.0265862941741943,-0.35325539112091064],                                                                                                                
[0.1918039619922638,0.06385798752307892,-0.07917416840791702],  
[0.21313326060771942,-0.26248422265052795,-0.20247593522071838],                                                                                                                
[0.21976569294929504,-0.5019633769989014,-0.3409276604652405], 
[0.19843746721744537,-0.9557111859321594,-0.39101099967956543],                                                                                                                 
[0.39327263832092285,0.2001791000366211,-0.18606005609035492], 
[0.5203642845153809,-0.1372901201248169,-0.288455605506897],                                                                                                                    
[0.6039697527885437,-0.3297727108001709,-0.37172359228134155], 
[0.6639994978904724,-0.6602937579154968,-0.47589367628097534],

### 50-30
# [0.012122005224227905,0.9222052097320557,0.305247038602829],  
# [-0.3017532229423523,0.6506431698799133,0.15650737285614014],
# [-0.5491971969604492,0.4329034090042114,-0.08598979562520981], 
# [-0.7254341244697571,0.09015665203332901,-0.22999104857444763],
# [-0.9504801034927368,-0.2384749948978424,-0.2778995633125305],  
# [-0.2668836712837219,-0.04546454921364784,0.04236932098865509],  
# [-0.2985071539878845,-0.4259682893753052,-0.020225519314408302], 
# [-0.36887598037719727,-0.7100534439086914,-0.09502467513084412],
# [-0.3794710636138916,-1.0352394580841064,-0.13526804745197296], 
# [-6.437183008856096e-10,-2.173057289667213e-10,9.041620785410487e-10],   
# [-0.029111139476299286,-0.41261938214302063,-0.05721444636583328],
# [-0.009218394756317139,-0.7831172943115234,-0.09456558525562286],
# [0.09903229027986526,-1.2005475759506226,-0.02831835299730301],
# [0.1836147904396057,0.07991480827331543,-0.08142564445734024], 
# [0.27298158407211304,-0.270415335893631,-0.13706836104393005],  
# [0.33949849009513855,-0.5579192638397217,-0.2747378349304199],
# [0.3610542416572571,-0.953980565071106,-0.25432634353637695],  
# [0.40942418575286865,0.22639358043670654,-0.1937991976737976], 
# [0.5827551484107971,-0.12179119139909744,-0.24147416651248932],
# [0.6625240445137024,-0.2832615375518799,-0.3442637026309967],  
# [0.7498003244400024,-0.5539906620979309,-0.4425949454307556], 
### 51-56
# [0.04837186634540558,0.953267514705658,0.14152827858924866], 
# [-0.2134033888578415,0.6893223524093628,-0.026413418352603912], 
# [-0.44877320528030396,0.36373788118362427,-0.0980675145983696], 
# [-0.49661463499069214,0.0693720355629921,-0.1968698799610138],  
# [-0.5343084931373596,-0.22761213779449463,-0.32762476801872253],
# [-0.22808879613876343,0.01631726324558258,0.005199282430112362],  
# [-0.3275028169155121,-0.1724313199520111,-0.16341276466846466],  
# [-0.3740280270576477,0.04342292994260788,-0.3818708658218384],    
# [-0.23220330476760864,0.21193160116672516,-0.273222416639328],  
# [7.587237504935729e-10,-3.6215097587444234e-10,-1.1862828497299915e-09],
# [-0.06904375553131104,-0.2761349380016327,-0.20347920060157776], 
# [-0.1138867735862732,0.01699807681143284,-0.35628217458724976],   
# [-0.08915122598409653,0.2707883417606354,-0.33123594522476196], 
# [0.19179528951644897,0.05882798135280609,-0.04976610466837883], 
# [0.1482747346162796,-0.15043596923351288,-0.2604759931564331], 
# [0.07462721318006516,0.03589138761162758,-0.3592630624771118],   
# [0.04645075649023056,0.3275179862976074,-0.39163315296173096],
# [0.3863127827644348,0.13175179064273834,-0.18362154066562653], 
# [0.30512434244155884,-0.018150098621845245,-0.3896621763706207], 
# [0.24016064405441284,0.08922740072011948,-0.4608789086341858], 
# [0.21253953874111176,0.337272584438324,-0.4721159338951111],   
### 52-52
# [0.007394630461931229,0.9631226658821106,0.29846495389938354],
# [-0.25699836015701294,0.6931418180465698,-0.13346540927886963],
# [-0.2741612493991852,0.46609729528427124,-0.4512011408805847],  
# [-0.16915805637836456,0.28775328397750854,-0.6304642558097839],
# [0.0077946484088897705,-0.055119484663009644,-0.6765443682670593],
# [-0.26966559886932373,0.04490860551595688,-0.14783088862895966], 
# [-0.22646793723106384,-0.3954358398914337,-0.26482561230659485], 
# [-0.19393041729927063,-0.7292053699493408,-0.36724087595939636],
# [-0.16711348295211792,-1.0249546766281128,-0.43086743354797363],
# [1.3273635524058136e-09,-1.842682673114382e-09,-1.6879938558034269e-09],
# [0.04171294718980789,-0.14265578985214233,-0.340076208114624],  
# [0.05363078415393829,0.23801541328430176,-0.32466834783554077], 
# [0.08997689932584763,0.39975184202194214,-0.258650004863739],  
# [0.181292325258255,0.08663515001535416,0.044620074331760406],  
# [0.2199089676141739,0.03441227972507477,-0.2820286750793457],   
# [0.20033228397369385,0.21781256794929504,-0.4089891314506531],
# [0.20659412443637848,0.45560944080352783,-0.3331616520881653], 
# [0.4164486825466156,0.2312048375606537,-0.004265621304512024],  
# [0.44970738887786865,0.127219557762146,-0.2870674729347229],   
# [0.4225648045539856,0.22777768969535828,-0.39171528816223145], 
# [0.40110132098197937,0.4837588667869568,-0.18161074817180634],
### 53-45
# [-0.15657159686088562,0.8646677732467651,0.46411818265914917],
# [-0.3201183080673218,0.5978265404701233,0.04109055548906326],  
# [-0.2067355513572693,0.4720739424228668,-0.2674419581890106],  
# [-0.0752309113740921,0.32454797625541687,-0.4068257212638855],  
# [0.09519825875759125,0.15286190807819366,-0.5447385311126709],              
# [-0.2527685761451721,-0.0052520036697387695,-0.07545723766088486], 
# [-0.26122942566871643,-0.41103291511535645,-0.22101552784442902],          
# [-0.2667941749095917,-0.6172770261764526,-0.4103820025920868],        
# [-0.2162088304758072,-0.8242220878601074,-0.5256474614143372],                                                                                                                  
# [-1.0955626406072838e-09,1.2507271884842908e-09,-2.7882005593227177e-09],
# [0.09703614562749863,-0.4199373722076416,-0.18832048773765564],                                                                                                                 
# [0.2048678696155548,-0.5982843041419983,-0.3594641387462616],  
# [0.3008444309234619,-0.8220869898796082,-0.6155162453651428],                                                                                                                   
# [0.19363249838352203,0.10342378914356232,0.020827485248446465],
# [0.21263311803340912,0.05121283233165741,-0.28041940927505493],                                                                                                                 
# [0.10494331270456314,0.25853779911994934,-0.3873474597930908], 
# [0.024588726460933685,0.3473294973373413,-0.26178622245788574],                                                                                                                 
# [0.3285413682460785,0.29607534408569336,-0.019529195502400398], 
# [0.33031925559043884,0.2507806718349457,-0.3129718005657196],                                                                                                                   
# [0.23022347688674927,0.31269168853759766,-0.3426643908023834], 
# [0.14403969049453735,0.35934561491012573,-0.3381202518939972], 
  ])

else:
  human_points = np.array([
  ## right
  [0.35925376415252686,0.7557541131973267,0.5102125406265259],
  [-0.0217706561088562,0.7466740012168884,0.27697762846946716],   
  [-0.2586385905742645,0.6049586534500122,-0.09853920340538025], 
  [-0.2637333273887634,0.44316551089286804,-0.5239014029502869],  
  [-0.09836322069168091,0.3405230641365051,-0.8661985993385315],              
  [-0.26269078254699707,0.09757312387228012,-0.04891512915492058],
  [-0.40207889676094055,-0.3215048313140869,-0.14132839441299438],            
  [-0.5895003080368042,-0.5254322290420532,-0.2374584823846817],           
  [-0.6998971700668335,-0.7604447603225708,-0.34814760088920593],                                                                                                                 
  [1.2982748209822148e-10,-9.16070552747783e-10,8.97446561509696e-10],     
  [-0.08608043193817139,-0.422415554523468,-0.14894185960292816],                                                                                                                 
  [-0.18812528252601624,-0.68717360496521,-0.25021734833717346],  
  [-0.2508523464202881,-1.0579893589019775,-0.22055749595165253],                                                                                                                 
  [0.2520734667778015,0.018618403002619743,-0.008291996084153652],  
  [0.20300228893756866,-0.3212728202342987,-0.13540016114711761],                                                                                                                 
  [0.21015995740890503,-0.5265160202980042,-0.3273022770881653], 
  [0.09568844735622406,-0.9591401815414429,-0.41275525093078613],                                                                                                                 
  [0.5308778285980225,0.13585840165615082,-0.032896988093853],   
  [0.5607253313064575,-0.20470702648162842,-0.1921987533569336],                                                                                                                  
  [0.5806171298027039,-0.4187415838241577,-0.28038501739501953], 
  [0.5910985469818115,-0.7268421649932861,-0.4496389925479889], 
  ])

human_points_tmp = human_points.copy()
human_points_tmp = human_points_tmp - human_points_tmp[0, :]
thumb_base_t_wrist = se3.Transform(xyz=[24.0476665e-3, 3.78124745e-3, 32.32964923e-3]).inverse()
robot_t_human_base = se3.Transform(rot=np.radians([0, 0, 180])) * se3.Transform(
  xyz=(world_t_base * base_t_thumb_base * thumb_base_t_wrist).translation,
  rot=np.radians([-115, 0, 0])
)
human_points = geometry_utils.transform_points(human_points_tmp.transpose(),
                                                    robot_t_human_base).transpose() * 0.7 # scale to robot hand
human_vectors = human_points_to_vectors(human_points)

# def weight_and_scale(vector, vector_index):
#   length_threshold = 0.1 / unit_length
#   length = vector_length(vector)
#   antipodal_set = set([5, 6, 7, 8])
#   primary_set = set([9, 10, 11, 12, 13, 14])
#   if length > length_threshold:
#     return 1, 1.6 * length  # length #
#   elif vector_index in antipodal_set:
#     return 200, 1e-4  # length #
#   elif vector_index in primary_set:
#     return 400, 3e-2  # length #
#   else:
#     return 1, 1.6 * length

def weight_and_scale(vector, vector_index):
  vector_norm = vector_length(human_vectors[vector_index])
  antipodal_set = set([5, 6, 7, 8])
  primary_set = set([9, 10, 11, 12, 13, 14])
  if vector_index in antipodal_set:
    return 5, vector_norm
  elif vector_index in primary_set:
    return 10, vector_norm
  else:
    return 1, vector_norm


def metric(reduced_joint_positions, gradient):
  cost = 0
  num_dof = 16
  joint_positions = augment_reduced_joints(reduced_joint_positions)
  vectors = -np.array(robot_config_to_vectors(joint_positions))
  new_grad = np.zeros((num_dof, ))
  jacobians = compute_jacobians(joint_positions)
  for vector_index, vector in enumerate(vectors):
    weight, scale = weight_and_scale(vector, vector_index)
    diff = vector - scale * normalize(human_vectors[vector_index])
    cost += weight * np.linalg.norm(diff) ** 2
    d_vector_d_joint_position = np.zeros((3, num_dof))
    for tip_index in range(d_vector_d_tip_positions.shape[1]):
      d_vector_d_joint_position += d_vector_d_tip_positions[
        vector_index, tip_index] * jacobians[tip_index][:3, :]
    new_grad += weight * np.dot(diff, d_vector_d_joint_position)
  cost = 2.5e-3 * np.linalg.norm(joint_positions) ** 2 + 0.5 * cost
  new_grad += 5e-3 * joint_positions
  if gradient.size > 0:
    gradient[:] = new_grad[[6, 8, 10, 12, 14, 15]]
#   print(reduced_joint_positions)
#   print(cost)

  return cost

# GN_DIRECT
opt = nlopt.opt(nlopt.GN_DIRECT, 6)  # try different optimizers
opt.set_lower_bounds([0, 0, 0, 0, -2.0943951, 0])
opt.set_upper_bounds([2.0943951, 2.0943951, 2.0943951, 2.0943951, 0, 2.0943951])
opt.set_min_objective(metric)
opt.set_ftol_rel(1e-3)
opt.set_maxtime(1000)
theta = opt.optimize(np.array([1.0, 1.0, 1.0, 1.0, -1.0, 1.0]))
minf = opt.last_optimum_value()
print(theta)
print(minf)
print(opt.last_optimize_result())

[ 0.32320912  0.24563893  0.24563893  0.09049855 -0.01292836  0.76277352]
1.7493182987463742
3


In [18]:
jps = np.zeros((22))
jps[[7, 10, 13, 16, 19, 20]] = theta[[0, 1, 2, 3, 4, 5]]

jps[[8, 11, 14, 17]] = jps[[7, 10, 13, 16]] * 1.05851325 + 0.72349796
for i in range(22):
  p.resetJointState(robot_id, i, jps[i])

In [15]:
# debugging
plot_joints = augment_reduced_joints(theta)

tip_positions = []
for index, finger_tip_frame in enumerate(finger_tips):
  finger_joints = [plot_joints[joint_indices[joint_name]]
                   for joint_name in tip_joints[finger_tip_frame]]
  tip_positions.append(finger_chains[index].forward_kinematics(finger_joints).pos)
base_t_xyz = np.array([
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  tip_positions[0],
  tip_positions[0],
  tip_positions[0],
  tip_positions[0],
  tip_positions[1],
  tip_positions[1],
  tip_positions[1],
  tip_positions[2],
  tip_positions[2],
  tip_positions[3],
])
start_xyz = geometry_utils.transform_points(
  base_t_xyz.transpose(), world_t_base * base_t_thumb_base).transpose()
diff_xyz = np.array(robot_config_to_vectors(plot_joints)) * unit_length
end_xyz = geometry_utils.transform_points(
  (base_t_xyz + diff_xyz).transpose(), world_t_base * base_t_thumb_base).transpose()

for i in range(0, start_xyz.shape[0]):
  p.addUserDebugLine(start_xyz[i, :], end_xyz[i, :], lineColorRGB=[0, 0, 1], lifeTime=30)

base_t_xyz = human_points[[0, 0, 0, 0, 0, 4, 4, 4, 4, 8, 8, 8, 12, 12, 16], :] * unit_length
start_xyz = base_t_xyz
diff_xyz = np.array(human_vectors) * unit_length
end_xyz = base_t_xyz + diff_xyz

for i in range(0, start_xyz.shape[0]):
  p.addUserDebugLine(start_xyz[i, :], end_xyz[i, :], lineColorRGB=[1, 0, 0], lifeTime=30)

In [20]:
p.disconnect()

In [19]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
colors = (np.array([list(range(20, -1, -1)), [0] * 21, list(range(0, 21))]).transpose() / 20)
colors = ['k'] + ['r'] * 4 + ['g'] * 4 + ['b'] * 4 + ['y'] * 4 + ['c'] * 4
i = 0
ax.scatter(human_points[i,0],human_points[i,1],human_points[i,2],color=colors[i])
ax.text(human_points[i,0],human_points[i,1],human_points[i,2],  '%s' % (str(i)), size=10, zorder=1,color='k')
for i in range(5):
  for j in range(4):
    ax.scatter(human_points[i * 4 + j+1,0],human_points[i * 4 + j+1,1],human_points[i*4 + j+1,2],color=colors[i*4 + j+1])
    if j == 0:
      ax.plot([human_points[i * 4 + j+1,0], human_points[0,0]],
              [human_points[i * 4 + j+1,1], human_points[0,1]],
              [human_points[i*4 + j+1,2], human_points[0,2]],color=colors[i*4 + j+1]
              )
    else:
      ax.plot([human_points[i * 4 + j,0], human_points[i * 4 + j + 1,0]],
              [human_points[i * 4 + j,1], human_points[i * 4 + j + 1,1]],
              [human_points[i*4 + j,2], human_points[i*4 + j + 1,2]],color=colors[i*4 + j+1]
              )
    ax.text(human_points[i*4+j+1,0],human_points[i*4+j+1,1],human_points[i*4+j+1,2],  '%s' % (str(i*4+j+1)), size=10, zorder=1,color='k')
#   ax.scatter(human_points[i,0],human_points[i,1],human_points[i,2],color=colors[i])
#   ax.text(human_points[i,0],human_points[i,1],human_points[i,2],  '%s' % (str(i)), size=10, zorder=1,color='k')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
ax.set_zlim([-1,1])

(-1.0, 1.0)